# Plot summary generator

In [ ]:
## adding directory to system path --- execute only once
#import sys,os
#os.chdir('C:\\Users\\Rohit Suresh\\Jupyter projects\\Plot summary generator')
#sys.path.append(os.getcwd())
#sys.path

In [1]:
# import necessary packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
import inspect
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
import importlib
import plot_summary_generator_utils
importlib.reload(plot_summary_generator_utils)
from plot_summary_generator_utils import *
import warnings
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec,KeyedVectors

<IPython.core.display.Javascript object>

In [ ]:
# see that all your functions from utils have been imported
all_functions = inspect.getmembers(plot_summary_generator_utils , inspect.isfunction)
for fn , _ in all_functions: print(str(fn))

# Train embedding matrix
## 1. Obtain plot summaries

Plot summaries will be obtained from Wikipedia. We will use the 'wikipedia' package to operate on wikipedia pages.

In [ ]:
filepath = 'corpus_movies_list.xlsx'
imdb_ids = imdb_id_list(filepath)
imdb_ids

In [ ]:
%time plot_aggregator_imdb(imdb_ids,'mcu_agg_plots') 

In [ ]:
with open('mcu_agg_plots.txt','r',encoding = 'utf-8') as f:
    mcu_corpus = f.read()

In [ ]:
mcu_corpus

In [ ]:
%time corpus_tokens = preprocess_corpus_text(mcu_corpus,lemmatize = False)

In [ ]:
corpus_tokens[-50:]

In [ ]:
#%time embed = Word2Vec(corpus_tokens, size = 300, min_count = 3, sg = 1, iter=200, negative=10)

In [ ]:
#print(embed)

In [ ]:
#embed_words = list(embed.wv.vocab)
#print(embed_words[0:25])

In [ ]:
#print(embed.wv['thanos'])

In [ ]:
#word_vectors = embed.wv

In [ ]:
#res = word_vectors.most_similar(positive=['wasp','man'],negative=['woman'])

In [ ]:
#res[:3]

In [ ]:
#res1 = word_vectors.most_similar(positive=['barton','woman'],negative=['man'])

In [ ]:
#res1[:3]

In [ ]:
#res2 =word_vectors.most_similar(positive=['thor','tony'],negative=['odin'])

In [ ]:
#res2[:3]

In [ ]:
#res3 = word_vectors.similar_by_word('peter')

In [ ]:
#res3[:3]

In [ ]:
cl = pd.read_csv('comic_titles.csv',header=0,encoding = 'unicode_escape')['comic_title']

In [ ]:
s = sample(list(cl),1000)
s

In [ ]:
%%notify
#%time comic_plot_agg(s,'corpus_1000_random')

In [ ]:
#with open('corpus_1000_random.txt','r',encoding = 'utf-8') as f: corpus_raw_text = f.read()

In [ ]:
%%notify
#%time corpus_tokens = preprocess_corpus_text(corpus_raw_text)

In [ ]:
%%notify
#%time embed = Word2Vec(corpus_tokens, size = 100, min_count = 3, sg = 1, iter=300, negative=10)

In [ ]:
#embed.wv.save_word2vec_format('1000_comic_model')

In [ ]:
word_embed = KeyedVectors.load_word2vec_format('1000_comic_model')

In [ ]:
word_embed.wv.similar_by_word('johnny')

In [ ]:
word_embed.wv.most_similar(positive = ['johnny','woman'], negative = ['man'])

In [ ]:
word_embed.wv.similar_by_word('thanos')

In [ ]:
word_embed.wv.similar_by_word('galactus')

In [ ]:
word_embed.wv.similar_by_word('thor')

In [ ]:
word_embed.wv.similar_by_word('asgard')

In [ ]:
word_embed.wv.similar_by_word('loki')

In [ ]:
word_embed.wv.most_similar(positive = ['thor','beta','ray','bill'], negative = ['mjolnir'])

In [ ]:
word_embed.wv.most_similar(positive = ['thor','loki'], negative = ['odin'])

In [ ]:
word_embed.wv.most_similar(positive = ['clint','woman'], negative = ['man'])

In [ ]:
word_embed.wv.similar_by_word('namor')

## Testing out generator

In [221]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
import keras.utils as KU
import gensim
from itertools import chain
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, Dropout, BatchNormalization
from random import randint
from keras.preprocessing.sequence import pad_sequences
import keras.backend as KB
from keras.initializers import Constant
from keras.callbacks import EarlyStopping

In [ ]:
vocab = gen_embed.wv.vocab

In [ ]:
all_words = list(vocab)

In [ ]:
t = Tokenizer()
vocab_size = len(all_words) + 1
t.fit_on_texts(all_words)

In [108]:
def get_weight_matrix(embedding):
    vocab = embedding.wv.vocab
    all_words = list(vocab)
    vocab_size = len(all_words) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size,embedding.vector_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for i in range(len(all_words)):
        weight_matrix[i + 1] = embedding[all_words[i]]
    return weight_matrix

In [3]:
text = plot_puller_imdb('4154796')

In [4]:
tokens = preprocess_corpus_text(text,lemmatize = False,remove_stopwords = False)

In [5]:
tokens

[['after',
  'thanos',
  'activates',
  'the',
  'completed',
  'infinity',
  'gauntlet',
  'clint',
  'bartons',
  'family',
  'disintegrates',
  'along',
  'with',
  'half',
  'of',
  'all',
  'life',
  'across',
  'the',
  'universe'],
 ['nebula',
  'and',
  'tony',
  'stark',
  'are',
  'stranded',
  'in',
  'space',
  'following',
  'their',
  'defeat',
  'by',
  'thanos',
  'on',
  'titan',
  'but',
  'are',
  'returned',
  'to',
  'earth',
  'by',
  'carol',
  'danvers',
  'and',
  'reunited',
  'with',
  'natasha',
  'romanoff',
  'bruce',
  'banner',
  'steve',
  'rogers',
  'rocket',
  'thor',
  'and',
  'james',
  'rhodes'],
 ['the',
  'team',
  'formulates',
  'a',
  'plan',
  'to',
  'steal',
  'the',
  'infinity',
  'stones',
  'back',
  'from',
  'thanos',
  'and',
  'use',
  'them',
  'to',
  'reverse',
  'his',
  'actions',
  'but',
  'learn',
  'upon',
  'finding',
  'him',
  'that',
  'he',
  'had',
  'used',
  'the',
  'stones',
  'a',
  'second',
  'time',
  'to',


In [145]:
%%notify
gen_embed = Word2Vec(tokens, size = 300, min_count = 1, sg = 1, iter=200, negative=10)

<IPython.core.display.Javascript object>

In [146]:
print(gen_embed)

Word2Vec(vocab=559, size=300, alpha=0.025)


In [147]:
embed_mtx = get_weight_matrix(gen_embed)

In [148]:
vocab_size = len(gen_embed.wv.vocab)

In [149]:
embed_size = gen_embed.vector_size

In [6]:
token_array = []
for sen in tokens:
    for word in sen:
        token_array.append(word)
token_array

['after',
 'thanos',
 'activates',
 'the',
 'completed',
 'infinity',
 'gauntlet',
 'clint',
 'bartons',
 'family',
 'disintegrates',
 'along',
 'with',
 'half',
 'of',
 'all',
 'life',
 'across',
 'the',
 'universe',
 'nebula',
 'and',
 'tony',
 'stark',
 'are',
 'stranded',
 'in',
 'space',
 'following',
 'their',
 'defeat',
 'by',
 'thanos',
 'on',
 'titan',
 'but',
 'are',
 'returned',
 'to',
 'earth',
 'by',
 'carol',
 'danvers',
 'and',
 'reunited',
 'with',
 'natasha',
 'romanoff',
 'bruce',
 'banner',
 'steve',
 'rogers',
 'rocket',
 'thor',
 'and',
 'james',
 'rhodes',
 'the',
 'team',
 'formulates',
 'a',
 'plan',
 'to',
 'steal',
 'the',
 'infinity',
 'stones',
 'back',
 'from',
 'thanos',
 'and',
 'use',
 'them',
 'to',
 'reverse',
 'his',
 'actions',
 'but',
 'learn',
 'upon',
 'finding',
 'him',
 'that',
 'he',
 'had',
 'used',
 'the',
 'stones',
 'a',
 'second',
 'time',
 'to',
 'destroy',
 'them',
 'preventing',
 'their',
 'further',
 'use',
 'in',
 'anger',
 'thor',
 '

In [150]:
length = 5 + 1
sequences = list()
for i in range(length, len(token_array)):
    # select sequence of tokens
    seq = token_array[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 1544


In [151]:
sequences[:2]

['after thanos activates the completed infinity',
 'thanos activates the completed infinity gauntlet']

In [152]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
tok_seq = tokenizer.texts_to_sequences(sequences)

In [153]:
vocab_size = len(tokenizer.word_index) + 1

In [154]:
# separate into input and output
tok_seq = np.array(tok_seq)
X, y = tok_seq[:,:-1], tok_seq[:,-1]
y = KU.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [273]:
model = Sequential()
model.add(Embedding(vocab_size, embed_size, embeddings_initializer=Constant(embed_mtx),input_length=5,trainable=False,name = 'Endgame_embed'))
model.add(Bidirectional(LSTM(100, return_sequences=True,go_backwards=True, recurrent_dropout = 0.3),name='BiDi_LSTM_1'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100,go_backwards=True, recurrent_dropout = 0.3), name = 'Bidi_LSTM_2'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu',name='FC_1'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu',name='FC_2'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax',name='output_layer'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Endgame_embed (Embedding)    (None, 5, 300)            168000    
_________________________________________________________________
BiDi_LSTM_1 (Bidirectional)  (None, 5, 200)            320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 200)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 200)            800       
_________________________________________________________________
Bidi_LSTM_2 (Bidirectional)  (None, 200)               240800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 200)               800       
__________

In [274]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [276]:
early_stopper = EarlyStopping(monitor = 'val_acc', patience = 50, mode='min', verbose=1, restore_best_weights=True)

In [277]:
# fit model
model.fit(X, y, batch_size=32, epochs=75,verbose=1,callbacks=[early_stopper])

Epoch 1/75
1544/1544 [==============================] - 7s 4ms/step - loss: 6.4478 - acc: 0.0026
Epoch 2/75
1544/1544 [==============================] - 2s 1ms/step - loss: 6.2388 - acc: 0.0149 ETA: 1s - los
Epoch 3/75
1544/1544 [==============================] - 2s 1ms/step - loss: 6.0890 - acc: 0.0298
Epoch 4/75
1544/1544 [==============================] - 2s 2ms/step - loss: 5.8889 - acc: 0.0551
Epoch 5/75
1544/1544 [==============================] - 2s 2ms/step - loss: 5.6724 - acc: 0.0797
Epoch 6/75
1544/1544 [==============================] - 3s 2ms/step - loss: 5.4083 - acc: 0.0965
Epoch 7/75
1544/1544 [==============================] - 3s 2ms/step - loss: 5.1780 - acc: 0.1101
Epoch 8/75
1544/1544 [==============================] - 3s 2ms/step - loss: 4.9703 - acc: 0.1120
Epoch 9/75
1544/1544 [==============================] - 2s 2ms/step - loss: 4.7865 - acc: 0.1244
Epoch 10/75
1544/1544 [==============================] - 2s 1ms/step - loss: 4.6256 - acc: 0.1431
Epoch 11/75
154

In [247]:
seq_length = len(sequences[0].split()) - 1

In [248]:
# select a seed text
seed_text = sequences[randint(0,len(sequences))]
print(seed_text + '\n')

the quantum portal in the process



In [249]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]
enc = np.array([encoded[:-1]])

In [250]:
# predict probabilities for each word
yhat = model.predict_classes(enc, verbose=1)

1/1 [==============================] - 1s 656ms/step


In [251]:
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break

In [252]:
out_word

'process'

In [253]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [278]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

thanos reduces the avengers headquarters to rubble splitting the team up and causing the gauntlet to fall into bartons protection the past nebula is killed by her future self as she attempts to take the infinity stones from barton while rogers thor and stark confront thanos who decides he will


In [281]:
sdtx = 'you have full creative liberty'

In [282]:
generated1 = generate_seq(model, tokenizer, seq_length, sdtx, 500)
print(generated1)

rogers succeeds and the time stone stone after has it it from rocket and rhodes travel to the small town of new asgard where valkyrie and the last survivors of asgard have settled they there find thor who has become overwhelmed by guilt over not killing thanos the first time in wakanda thor has become majorly overweight his hair and beard are overgrown and he spends his free time eating junk food getting drunk and playing fortnite with his friends korg and miek thor begrudgingly agrees to return to the avengers after some convincing from rocket and banner after testing the quantum time machine on barton who confirms that it works the avengers are reunited with a plan banner rogers lang and stark embark to retrieve the time mind and space stones during the battle of new york in 2012 banner goes to the sanctum sanctorum where he is informed by the ancient one that taking the time stone from her time line would prevent stephen stranges future efforts to stop kaecilius from destroying the 

In [272]:
#KB.clear_session()